# Imports

In [5]:
import pandas as pd
import re

# Macros

In [6]:
DATA_FOLDER = '../../data/'
FIG_FOLDER = '../../figures/'

In [ ]:
3

# Utile Function

# Cleaning

## Inital Data import

In [8]:
df = pd.read_csv(DATA_FOLDER + 'loans_2007.csv')
OrgRowCount = df.shape[0]
OrgColCount = df.shape[1]

/Users/arnescheunemann/opt/anaconda3/envs/nf2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## dealing with NAN and numbers in strings 

In [9]:
# drop dublicated last rows seem to be sum of above and None lines
df = df.drop_duplicates()
## there seems to 3 with many empty cells so this is fine
df = df.dropna(thresh=50,axis=0)
df = df[~df.revol_util.isna()]

In [10]:
# transfer string 'num%' to number 0-1
df.revol_util = df.revol_util.apply(lambda x: str(x).replace('%','')).astype('float64')/100
df.int_rate = df.int_rate.apply(lambda x: str(x).replace('%','')).astype('float64')/100

In [ ]:
# emp_length transfrom form string to number (<1 years ,  years, 10+ years etc)
def Trans_empValue(s):
    if type(s) is float:
        return 0.
    if re.search('\<',s):
        return 0.5
    else:
        return float(re.search('\d+',s).group())

df.emp_length = df.emp_length.apply(lambda x: Trans_empValue(x))

In [ ]:
# no info about bakrupcie will be interpreted as none
df.pub_rec_bankruptcies = df.pub_rec_bankruptcies.fillna(0.0).astype('int32')
# cap the very view that have amount of 2 ( could be spouses etc...)
df.pub_rec_bankruptcies = df.pub_rec_bankruptcies.apply(lambda x: 1 if x > 0 else 0)
df.pub_rec_bankruptcies.value_counts()

## Export Cleaned Data 

In [ ]:
sa